In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install faker transformers accelerate -q

import pandas as pd
import random
import torch
from faker import Faker
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from torch.utils.data import Dataset
import matplotlib.pyplot as plt

# Configura Faker para dados brasileiros reais
fake = Faker('pt_BR')
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Ambiente pronto no: {device}")

Ambiente pronto no: cuda


In [4]:
print("Iniciando Geração Massiva de Dados Hierárquicos")
def aplicar_politica(tipo, valor):
    
    if tipo in ['cpf', 'senha', 'telefone', 'cartao', 'rg']:
        return "[REMOVIDO]"
    
    
    if tipo == 'nome': return "[NOME]"
    if tipo == 'email': return "[EMAIL]"
    
    
    return valor 


Iniciando Geração Massiva de Dados Hierárquicos


In [5]:
templates_banco = [
    "O cliente {nome} (CPF {cpf}) realizou um saque em {cidade}.",
    "Bloqueio preventivo do cartão {cartao} pertencente a {nome}.",
    "A gerente {gerente} autorizou o aumento de limite para {nome}.",
    "Transação suspeita: {nome} tentou usar o cartão {cartao} em {cidade}.",
    "Favor atualizar o cadastro de {nome}. Novo telefone: {telefone}."
]

templates_hospital = [
    "Paciente {nome}, residente em {cidade}, diagnosticado com gripe.",
    "O médico {medico} solicitou exames para {nome} (RG {rg}).",
    "Encaminhar resultado para o email {email} urgente.",
    "Paciente {nome} internado na UTI do hospital de {cidade}.",
    "Ligar para o responsável de {nome} no número {telefone}."
]

templates_rh = [
    "O funcionário {nome} foi promovido a {cargo} na filial de {cidade}.",
    "Login de rede criado para {nome}. Senha temporária: {senha}.",
    "Candidato {nome} aprovado para a vaga de {cargo}.",
    "Enviar holerite para o email corporativo: {email}.",
    "O colaborador {nome} esqueceu o crachá na portaria."
]

templates_juridico = [
    "Processo movido por {nome} contra a empresa de {cidade}.",
    "O réu {nome}, portador do CPF {cpf}, nega as acusações.",
    "A advogada {advogada} solicitou a revisão da pena de {nome}.",
    "Intimação enviada para o endereço de {nome} em {cidade}.",
    "Audiência de conciliação entre {nome} e a parte contrária."
]

In [6]:
data_pairs = []
num_exemplos = 20000 

print(f"Gerando {num_exemplos} frases.")

for _ in range(num_exemplos):
    dados = {
        'nome': fake.name(),
        'cpf': fake.cpf(),
        'rg': fake.rg(),
        'cidade': fake.city(),
        'telefone': fake.phone_number(),
        'email': fake.email(),
        'senha': fake.password(),
        'cartao': fake.credit_card_number(),
        'cargo': fake.job(),
        'medico': "Dr. " + fake.first_name(),
        'gerente': fake.first_name(),
        'advogada': "Dra. " + fake.first_name()
    }
    
    contexto = random.choice(['banco', 'hospital', 'rh', 'juridico'])
    
    if contexto == 'banco': template = random.choice(templates_banco)
    elif contexto == 'hospital': template = random.choice(templates_hospital)
    elif contexto == 'rh': template = random.choice(templates_rh)
    else: template = random.choice(templates_juridico)
    
    
    input_text = template.format(**dados)
    
    
    dados_tratados = {k: aplicar_politica(k, v) for k, v in dados.items()}
    target_text = template.format(**dados_tratados)
    
    data_pairs.append([f"anonimizar: {input_text}", target_text])


Gerando 20000 frases.


In [7]:
df = pd.DataFrame(data_pairs, columns=['input_text', 'target_text'])
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

print(f"Dataset com {len(df)} frases gerado.")
print("\n Amostra de dados e hierarquia")
pd.set_option('display.max_colwidth', None)

print(df.sample(5))

Dataset com 20000 frases gerado.

 Amostra de dados e hierarquia
                                                                                            input_text  \
3962                         anonimizar: Candidato Caroline Melo aprovado para a vaga de Endodontista.   
3434             anonimizar: Intimação enviada para o endereço de Giovanna Ramos em Fernandes da Mata.   
13488  anonimizar: Intimação enviada para o endereço de Isabela Casa Grande em Marques de Albuquerque.   
9942                    anonimizar: Intimação enviada para o endereço de Luna Moraes em Santos Alegre.   
14083                anonimizar: O médico Dr. Lucca solicitou exames para Camila Alves (RG 125834603).   

                                                                  target_text  
3962                   Candidato [NOME] aprovado para a vaga de Endodontista.  
3434        Intimação enviada para o endereço de [NOME] em Fernandes da Mata.  
13488  Intimação enviada para o endereço de [NOME] em Marq

In [8]:
tokenizer = T5Tokenizer.from_pretrained("t5-small", legacy=False)

class HierarquiaDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=128): 
        self.data = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self): return len(self.data)
    def __getitem__(self, index):
        row = self.data.iloc[index]
        inputs = self.tokenizer(row['input_text'], max_length=self.max_len, padding="max_length", truncation=True, return_tensors="pt")
        targets = self.tokenizer(row['target_text'], max_length=self.max_len, padding="max_length", truncation=True, return_tensors="pt")
        return {'input_ids': inputs['input_ids'].flatten(), 'attention_mask': inputs['attention_mask'].flatten(), 'labels': targets['input_ids'].flatten()}

train_dataset = HierarquiaDataset(train_df, tokenizer)
val_dataset = HierarquiaDataset(val_df, tokenizer)
print("Tokenização feita")

Tokenização feita


In [9]:
model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)

training_args = Seq2SeqTrainingArguments(
    output_dir="./resultado_hierarquia",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=3e-4,
    weight_decay=0.01,
    logging_steps=200,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    predict_with_generate=True,
    fp16=True,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
    tokenizer=tokenizer,
)

trainer.train()

model.save_pretrained("./modelo_final_tcc")
tokenizer.save_pretrained("./modelo_final_tcc")

/tmp/ipykernel_456/244735477.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input ten

Epoch,Training Loss,Validation Loss
1,0.150900,0.000000
2,0.000100,0.000000
3,0.000100,0.000000
4,0.000000,0.000000
5,0.000000,0.000000


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


('./modelo_final_tcc/tokenizer_config.json',
 './modelo_final_tcc/special_tokens_map.json',
 './modelo_final_tcc/spiece.model',
 './modelo_final_tcc/added_tokens.json')

In [15]:
import torch
import pandas as pd

model_final = T5ForConditionalGeneration.from_pretrained("./modelo_final_tcc").to(device)
tokenizer_final = T5Tokenizer.from_pretrained("./modelo_final_tcc", legacy=False)

def anonimizar(texto):
    inputs = tokenizer_final(f"anonimizar: {texto}", return_tensors="pt").input_ids.to(device)
    outputs = model_final.generate(inputs, max_length=128, num_beams=2, early_stopping=True)
    return tokenizer_final.decode(outputs[0], skip_special_tokens=True)

print("\n" + "="*80)

NameError: name 'T5ForConditionalGeneration' is not defined

In [14]:
testes = [
    ("FÁCIL", "O cliente João Silva tem o CPF 111.222.333-44."),
    ("FÁCIL", "A funcionária Maria Oliveira mora em Curitiba."),
    ("FÁCIL", "Login: admin. Senha: password123."),
    ("FÁCIL", "Favor enviar email para suporte@empresa.com.br."),

    
    ("MÉDIO", "Meu CPF é 04925133330 (sem pontuação)."),  
    ("MÉDIO", "Ligar para (19) 99888-7777 urgente."),       
    ("MÉDIO", "O Dr. Carlos atende no Hospital Sírio-Libanês."), 
    ("MÉDIO", "Acesso liberado para roberto.santos@gmail.com."), 
    
    
    ("DIFÍCIL", "O Pedro foi promovido."),                  
    ("DIFÍCIL", "A Rosa comprou flores."),                  
    ("DIFÍCIL", "Falar com o Leão."),                       
    ("DIFÍCIL", "Eu sou Arlyson."),                         
    
   
    ("HARDCORE", "Lebron James joga no Los Angeles Lakers."), 
    ("HARDCORE", "O juiz Sergio Moro e o ex-presidente Lula."), 
    ("HARDCORE", "A empresa Ford demitiu o funcionário Ford."), 
    ("HARDCORE", "Meu nome é Null e meu CPF é zero."),      
]

resultados = []

for nivel, frase in testes:
    saida = anonimizar(frase)
    print(f"[{nivel}]")
    print(f"Entrada: {frase}")
    print(f"Saída:   {saida}")
    
    
    resultados.append({'Nível': nivel, 'Entrada': frase, 'Saída': saida})


NameError: name 'anonimizar' is not defined

In [13]:
import torch

model_final = T5ForConditionalGeneration.from_pretrained("./modelo_final_tcc").to(device)
tokenizer_final = T5Tokenizer.from_pretrained("./modelo_final_tcc", legacy=False)

def anonimizar_frase(texto):
    entrada = f"anonimizar: {texto}"
    inputs = tokenizer_final(entrada, return_tensors="pt").input_ids.to(device)
    
    outputs = model_final.generate(
        inputs, 
        max_length=128, 
        num_beams=2,       
        early_stopping=True
    )
    
    # 3. Decodifica
    return tokenizer_final.decode(outputs[0], skip_special_tokens=True)

print("Página de Anonimização")
print("Digite uma frase e aperte ENTER para ver o resultado.")
print("Digite 'SAIR' para encerrar.")
print("="*60)

while True:
    try:
        frase_usuario = input("\nDigite a frase: ")
        
        
        if frase_usuario.lower() in ['sair', 'exit', 'pare']:
            print("\nEncerrando sistema!")
            break
        
        if len(frase_usuario.strip()) == 0:
            continue

        
        resultado = anonimizar_frase(frase_usuario)
        
        
        print("-" * 60)
        print(f"Apos a anonimização:")
        print(f" {resultado}")
        
        
    except KeyboardInterrupt:
        print("\n Sistema interrompido.")
        break
    except Exception as e:
        print(f"Erro: {e}")

Página de Anonimização
Digite uma frase e aperte ENTER para ver o resultado.
Digite 'SAIR' para encerrar.



Digite a frase:  ok


Erro: name 'anonimizar_frase' is not defined



Digite a frase:  sair



Encerrando sistema!
